# 🏪 Woodland Play Cafe - Membership Analysis Report

**Analysis Period:** August 2025  
**Purpose:** Comprehensive membership transaction and tax report analysis  
**Key Focus:** Membership revenue, subscription patterns, and tax reconciliation

## 📊 Analysis Overview
- **Transaction Records:** 1,926 total
- **Tax Records:** 918 total  
- **Membership Records:** 76 total
- **Membership Transactions:** Filtered by Order ID starting with 'MEM'


In [66]:
# ==============================
# 📦 Import Libraries
# ==============================
import pandas as pd
import numpy as np
from datetime import datetime

# ==============================
# 📂 File Paths
# ==============================
base_path = "/Users/vijayaraghavandevaraj/Library/Mobile Documents/com~apple~CloudDocs/Common/WoodLandPlayCafeAnalysis/MemebershipData"

trans_file = f"{base_path}/Transv1.xlsx"
tax_file = f"{base_path}/Tax.xlsx"
membership_file = f"{base_path}/Memebership.xlsx"

# ==============================
# 📥 Load Excel Files
# ==============================
print("📁 Loading Excel files...")

WoodTrans = pd.read_excel(trans_file)
WoodTax = pd.read_excel(tax_file)
SourceMembership = pd.read_excel(membership_file)

print(f"✅ Transaction Report loaded: {len(WoodTrans)} records")
print(f"✅ Tax Report loaded: {len(WoodTax)} records")
print(f"✅ Membership Report loaded: {len(SourceMembership)} records")

# ==============================
# 🧹 Clean Membership Data
# ==============================
Membership_df = SourceMembership[SourceMembership["membershipid"].notna()].copy()
print(f"✅ Clean Membership Records: {len(Membership_df)} records")

# ==============================
# 📋 Data Structure Analysis
# ==============================
print("\n📋 DATA STRUCTURE ANALYSIS:")
print("Transaction columns:", list(WoodTrans.columns))
print("Tax columns:", list(WoodTax.columns))
print("Membership columns:", list(Membership_df.columns))


📁 Loading Excel files...
✅ Transaction Report loaded: 1926 records
✅ Tax Report loaded: 918 records
✅ Membership Report loaded: 76 records
✅ Clean Membership Records: 76 records

📋 DATA STRUCTURE ANALYSIS:
Transaction columns: ['Location', 'Order ID', 'Transaction Date', 'Amount', 'Source', 'Payment Type', 'Payment Gateway', 'Transaction Type', 'Card Type', 'Currency', 'Payment ID', 'Type']
Tax columns: ['Order ID', 'Module Name', 'Location', 'Month', 'Date', 'Year', 'Order Status', 'Payment Status', 'Tip', 'Tax', 'Total Sum', 'Total Sum.1']
Membership columns: ['membershipid', 'membershiptype', 'membershiptypeid', 'membershiptypename', 'orderstatus', 'paymentstatus', 'purchasedcustomerid', 'purchasedcustomername', 'purchasedon', 'startdate', 'enddate', 'isactive', 'document_id', 'document', 'membershiptypeprice', 'subtotal', 'tax', 'tip', 'total', 'discount', 'gratuitypercentage', 'location_id', 'location_name', 'subscriptionid', 'customerprofileid', 'paymentprofileid', 'subscriptions

In [82]:
# ==============================
# 🔍 STEP 1: Identify Membership Transactions
# ==============================
print("\n🔍 STEP 1: IDENTIFYING MEMBERSHIP TRANSACTIONS")
print("=" * 70)

# Create Module column based on Order ID prefix
WoodTrans["Module"] = np.where(
    WoodTrans["Order ID"].astype(str).str.startswith("MEM"), 
    "memberships", 
    WoodTrans["Source"]
)

# Filter only membership transactions
WoodTrans_memberships = WoodTrans[WoodTrans["Module"] == "memberships"].copy()

# ==============================
# 📊 Summary
# ==============================
total_txn = len(WoodTrans)
membership_txn = len(WoodTrans_memberships)
percentage = (membership_txn / total_txn * 100) if total_txn > 0 else 0

print("📊 MEMBERSHIP TRANSACTION SUMMARY:")
print(f"- Total transactions: {total_txn}")
print(f"- Membership transactions: {membership_txn}")
print(f"- Membership percentage: {percentage:.2f}%")

# ==============================
# 💰 Revenue Insights
# ==============================
if membership_txn > 0:
    total_revenue = WoodTrans_memberships["Amount"].sum()
    avg_amount = WoodTrans_memberships["Amount"].mean()

    print("\n💰 MEMBERSHIP REVENUE:")
    print(f"- Total membership revenue: ${total_revenue:,.2f}")
    print(f"- Average membership amount: ${avg_amount:,.2f}")

    # Show sample transactions
    print("\n📋 SAMPLE MEMBERSHIP TRANSACTIONS:")
    print(WoodTrans_memberships[["Order ID", "Transaction Date", "Amount", "Source"]].head())
else:
    print("\n❌ No membership transactions found!")



🔍 STEP 1: IDENTIFYING MEMBERSHIP TRANSACTIONS
📊 MEMBERSHIP TRANSACTION SUMMARY:
- Total transactions: 1926
- Membership transactions: 121
- Membership percentage: 6.28%

💰 MEMBERSHIP REVENUE:
- Total membership revenue: $11,904.02
- Average membership amount: $98.38

📋 SAMPLE MEMBERSHIP TRANSACTIONS:
             Order ID Transaction Date  Amount   Source
119  MEM1757533524722       2025-09-10  130.60      crm
258  MEM1757170773220       2025-09-06  130.60  website
298  MEM1754425853204       2025-09-05  207.43      pos
304  MEM1757013478756       2025-09-04  130.60  website
332  MEM1756923580036       2025-09-03  130.60  website


In [68]:
# ==============================
# 🔗 STEP: Merge Membership Start Dates
# ==============================

# Merge membership startdate into membership transactions
WoodTrans_memberships = WoodTrans_memberships.merge(
    SourceMembership[["membershipid", "startdate"]],
    left_on="Order ID",
    right_on="membershipid",
    how="left"
)

# Drop duplicate membershipid column (since Order ID already exists)
if "membershipid" in WoodTrans_memberships.columns:
    WoodTrans_memberships = WoodTrans_memberships.drop(columns=["membershipid"])

# Convert startdate to datetime for consistency
WoodTrans_memberships["startdate"] = pd.to_datetime(
    WoodTrans_memberships["startdate"], errors="coerce"
)

# Preview the enriched dataset
print("\n📋 ENRICHED MEMBERSHIP TRANSACTIONS SAMPLE:")
print(WoodTrans_memberships[["Order ID", "Transaction Date", "Amount", "Source", "startdate"]].head())



📋 ENRICHED MEMBERSHIP TRANSACTIONS SAMPLE:
           Order ID Transaction Date  Amount   Source  startdate
0  MEM1757533524722       2025-09-10  130.60      crm 2025-09-10
1  MEM1757170773220       2025-09-06  130.60  website 2025-09-06
2  MEM1754425853204       2025-09-05  207.43      pos 2025-08-05
3  MEM1757013478756       2025-09-04  130.60  website 2025-09-04
4  MEM1756923580036       2025-09-03  130.60  website 2025-09-03


In [69]:
import pandas as pd

# Ensure Transaction Date is datetime
WoodTrans_memberships["Transaction Date"] = pd.to_datetime(
    WoodTrans_memberships["Transaction Date"], errors="coerce"
)

# Filter for August 2025
august_2025_data = WoodTrans_memberships[
    (WoodTrans_memberships["Transaction Date"].dt.month == 8) &
    (WoodTrans_memberships["Transaction Date"].dt.year == 2025)
].copy()

# Show results
print(f"✅ Found {len(august_2025_data)} membership transactions for August 2025")
print("\n📋 SAMPLE DATA (first 10 rows):")
print(august_2025_data.head(10))


✅ Found 32 membership transactions for August 2025

📋 SAMPLE DATA (first 10 rows):
          Location          Order ID Transaction Date  Amount   Source  \
9   East Nashville  MEM1753899349535       2025-08-31   87.80      crm   
10  East Nashville  MEM1753966592596       2025-08-31  152.55  website   
11  East Nashville  MEM1756566329830       2025-08-30  207.43  website   
12  East Nashville  MEM1756561841996       2025-08-30  130.60  website   
13  East Nashville  MEM1753899349535       2025-08-30  240.35  website   
14  East Nashville  MEM1753796245426       2025-08-29  130.60      crm   
15  East Nashville  MEM1756334515636       2025-08-27  152.55  website   
16  East Nashville  MEM1753534775988       2025-08-26  130.60  website   
17             NaN  MEM1753463742302       2025-08-25  139.00  website   
18  East Nashville  MEM1753462253033       2025-08-25  152.55      crm   

     Payment Type Payment Gateway Transaction Type        Card Type Currency  \
9     virtualCard     

In [70]:
# ==============================
# 🧹 Clean startdate & Mark Membership Type
# ==============================

# Ensure startdate is datetime
august_2025_data["startdate"] = pd.to_datetime(
    august_2025_data["startdate"], errors="coerce"
)

# Mark New vs Recurring
august_2025_data["Membership Type"] = august_2025_data["startdate"].apply(
    lambda x: "New" if pd.notnull(x) and (x.month == 8 and x.year == 2025) else "Recurring"
)

# Preview results
print("\n📋 SAMPLE CLASSIFICATION (first 10 rows):")
print(august_2025_data[["Order ID", "startdate", "Membership Type"]].head(10))



📋 SAMPLE CLASSIFICATION (first 10 rows):
            Order ID  startdate Membership Type
9   MEM1753899349535 2025-07-30       Recurring
10  MEM1753966592596 2025-07-31       Recurring
11  MEM1756566329830 2025-08-30             New
12  MEM1756561841996 2025-08-30             New
13  MEM1753899349535 2025-07-30       Recurring
14  MEM1753796245426 2025-07-29       Recurring
15  MEM1756334515636 2025-08-27             New
16  MEM1753534775988 2025-07-26       Recurring
17  MEM1753463742302 2025-07-25       Recurring
18  MEM1753462253033 2025-07-25       Recurring


In [71]:
# ==============================
# 📅 Filter Membership Transactions for August & August 2025 Credits
# ==============================

# Ensure Transaction Date is datetime
WoodTrans_memberships["Transaction Date"] = pd.to_datetime(
    WoodTrans_memberships["Transaction Date"], errors="coerce"
)

# All August transactions (any year)
august_data = WoodTrans_memberships[
    WoodTrans_memberships["Transaction Date"].dt.month == 8
].copy()

print(f"✅ Found {len(august_data)} total transactions in August (all years)")

# August 2025 + credit type
august_2025 = WoodTrans_memberships[
    (WoodTrans_memberships["Transaction Date"].dt.month == 8) &
    (WoodTrans_memberships["Transaction Date"].dt.year == 2025) &
    (WoodTrans_memberships["Type"].str.lower() == "credit")
].copy()

print(f"✅ Found {len(august_2025)} membership transactions in August 2025 (credit only)")
print("\n📋 SAMPLE DATA (first 10 rows):")
print(august_2025.head(10))


✅ Found 32 total transactions in August (all years)
✅ Found 29 membership transactions in August 2025 (credit only)

📋 SAMPLE DATA (first 10 rows):
          Location          Order ID Transaction Date  Amount   Source  \
10  East Nashville  MEM1753966592596       2025-08-31  152.55  website   
11  East Nashville  MEM1756566329830       2025-08-30  207.43  website   
12  East Nashville  MEM1756561841996       2025-08-30  130.60  website   
13  East Nashville  MEM1753899349535       2025-08-30  240.35  website   
14  East Nashville  MEM1753796245426       2025-08-29  130.60      crm   
15  East Nashville  MEM1756334515636       2025-08-27  152.55  website   
16  East Nashville  MEM1753534775988       2025-08-26  130.60  website   
17             NaN  MEM1753463742302       2025-08-25  139.00  website   
18  East Nashville  MEM1753462253033       2025-08-25  152.55      crm   
19  East Nashville  MEM1753455083253       2025-08-25  130.60      crm   

     Payment Type Payment Gateway Tra

In [72]:
# Count New vs Recurring
august_2025_counts = august_2025_data["Membership Type"].value_counts()

print("📊 Membership Type Distribution (August 2025)")
print("=" * 50)
for mtype, count in august_2025_counts.items():
    percentage = (count / len(august_2025_data)) * 100
    print(f"- {mtype}: {count} ({percentage:.2f}%)")


📊 Membership Type Distribution (August 2025)
- Recurring: 23 (71.88%)
- New: 9 (28.12%)


In [73]:
# ==============================
# 📋 Detailed Records by Membership Type
# ==============================

# Filter New and Recurring memberships
new_memberships = august_2025_data[august_2025_data["Membership Type"] == "New"].copy()
recurring_memberships = august_2025_data[august_2025_data["Membership Type"] == "Recurring"].copy()

# New memberships
print("=== NEW MEMBERSHIPS ===")
print(f"Total: {len(new_memberships)}\n")
print(new_memberships[["Order ID", "Location", "Transaction Date", "Amount", "startdate"]])

# Recurring memberships
print("\n=== RECURRING MEMBERSHIPS ===")
print(f"Total: {len(recurring_memberships)}\n")
print(recurring_memberships[["Order ID", "Location", "Transaction Date", "Amount", "startdate"]])


=== NEW MEMBERSHIPS ===
Total: 9

            Order ID        Location Transaction Date  Amount  startdate
11  MEM1756566329830  East Nashville       2025-08-30  207.43 2025-08-30
12  MEM1756561841996  East Nashville       2025-08-30  130.60 2025-08-30
15  MEM1756334515636  East Nashville       2025-08-27  152.55 2025-08-27
26  MEM1755711703484  East Nashville       2025-08-20  130.60 2025-08-20
35  MEM1755305206351  East Nashville       2025-08-15  152.55 2025-08-15
36  MEM1754596831519  East Nashville       2025-08-07  130.60 2025-08-07
37  MEM1754580996573  East Nashville       2025-08-07  152.55 2025-08-07
38  MEM1754425853204  East Nashville       2025-08-05  207.43 2025-08-05
39  MEM1754146369077  East Nashville       2025-08-02  130.60 2025-08-02

=== RECURRING MEMBERSHIPS ===
Total: 23

            Order ID        Location Transaction Date  Amount  startdate
9   MEM1753899349535  East Nashville       2025-08-31   87.80 2025-07-30
10  MEM1753966592596  East Nashville       2025-

In [74]:
# ==============================
# 🔗 Merge Tax Into August 2025 Data
# ==============================

# Ensure consistent data types
WoodTax["Order ID"] = WoodTax["Order ID"].astype(str)
august_2025["Order ID"] = august_2025["Order ID"].astype(str)

# Merge Tax column from WoodTax
august_2025 = august_2025.merge(
    WoodTax[["Order ID", "Tax"]],
    on="Order ID",
    how="left",
    suffixes=("", "_WoodTax")  # avoids accidental renaming conflicts
)

# Create Match_Status column
august_2025["Match_Status"] = august_2025["Tax"].apply(
    lambda x: "Match" if pd.notnull(x) else "Not Match"
)

# Preview enriched dataset
print("\n📋 SAMPLE AFTER TAX MERGE:")
print(august_2025[["Order ID", "Amount", "Location", "Transaction Date", "Tax", "Match_Status"]].head(10))



📋 SAMPLE AFTER TAX MERGE:
           Order ID  Amount        Location Transaction Date    Tax  \
0  MEM1753966592596  152.55  East Nashville       2025-08-31    NaN   
1  MEM1756566329830  207.43  East Nashville       2025-08-30  18.43   
2  MEM1756561841996  130.60  East Nashville       2025-08-30  11.60   
3  MEM1753899349535  240.35  East Nashville       2025-08-30    NaN   
4  MEM1753796245426  130.60  East Nashville       2025-08-29    NaN   
5  MEM1756334515636  152.55  East Nashville       2025-08-27  13.55   
6  MEM1753534775988  130.60  East Nashville       2025-08-26    NaN   
7  MEM1753463742302  139.00             NaN       2025-08-25    NaN   
8  MEM1753462253033  152.55  East Nashville       2025-08-25    NaN   
9  MEM1753455083253  130.60  East Nashville       2025-08-25    NaN   

  Match_Status  
0    Not Match  
1        Match  
2        Match  
3    Not Match  
4    Not Match  
5        Match  
6    Not Match  
7    Not Match  
8    Not Match  
9    Not Match  


In [75]:
# ==============================
# 🔗 Match Order IDs with Tax Report
# ==============================

# Ensure consistent types
WoodTax["Order ID"] = WoodTax["Order ID"].astype(str)
august_2025["Order ID"] = august_2025["Order ID"].astype(str)

# Flag Match vs Not Match
august_2025["Match_Status"] = august_2025["Order ID"].isin(WoodTax["Order ID"]).map(
    {True: "Match", False: "Not Match"}
)

# Show sample with match status
print("\n📋 SAMPLE MATCH CHECK:")
print(august_2025[["Order ID", "Match_Status"]].head(10))

# Summary counts
match_counts = august_2025["Match_Status"].value_counts()
print("\n📊 MATCH STATUS SUMMARY:")
for status, count in match_counts.items():
    percentage = (count / len(august_2025)) * 100
    print(f"- {status}: {count} ({percentage:.2f}%)")



📋 SAMPLE MATCH CHECK:
           Order ID Match_Status
0  MEM1753966592596    Not Match
1  MEM1756566329830        Match
2  MEM1756561841996        Match
3  MEM1753899349535    Not Match
4  MEM1753796245426    Not Match
5  MEM1756334515636        Match
6  MEM1753534775988    Not Match
7  MEM1753463742302    Not Match
8  MEM1753462253033    Not Match
9  MEM1753455083253    Not Match

📊 MATCH STATUS SUMMARY:
- Match: 22 (75.86%)
- Not Match: 7 (24.14%)


In [76]:
print(WoodTax.columns)
print(august_2025.columns)


Index(['Order ID', 'Module Name', 'Location', 'Month', 'Date', 'Year',
       'Order Status', 'Payment Status', 'Tip', 'Tax', 'Total Sum',
       'Total Sum.1'],
      dtype='object')
Index(['Location', 'Order ID', 'Transaction Date', 'Amount', 'Source',
       'Payment Type', 'Payment Gateway', 'Transaction Type', 'Card Type',
       'Currency', 'Payment ID', 'Type', 'Module', 'startdate', 'Tax',
       'Match_Status'],
      dtype='object')


In [77]:
# Ensure startdate is datetime
august_2025["startdate"] = pd.to_datetime(august_2025["startdate"], errors="coerce")

# Add Membership Type if missing
if "Membership Type" not in august_2025.columns:
    august_2025["Membership Type"] = august_2025["startdate"].apply(
        lambda x: "New" if pd.notnull(x) and (x.month == 8 and x.year == 2025) else "Recurring"
    )

# Now safe to preview with Membership Type
print(
    august_2025[
        ["Order ID", "Location", "Transaction Date", "Amount", "startdate", "Membership Type", "Tax", "Match_Status"]
    ].head()
)


           Order ID        Location Transaction Date  Amount  startdate  \
0  MEM1753966592596  East Nashville       2025-08-31  152.55 2025-07-31   
1  MEM1756566329830  East Nashville       2025-08-30  207.43 2025-08-30   
2  MEM1756561841996  East Nashville       2025-08-30  130.60 2025-08-30   
3  MEM1753899349535  East Nashville       2025-08-30  240.35 2025-07-30   
4  MEM1753796245426  East Nashville       2025-08-29  130.60 2025-07-29   

  Membership Type    Tax Match_Status  
0       Recurring    NaN    Not Match  
1             New  18.43        Match  
2             New  11.60        Match  
3       Recurring    NaN    Not Match  
4       Recurring    NaN    Not Match  


In [78]:
# ==============================
# 📊 Calculate Tax Percentage
# ==============================

# Calculate tax % safely
august_2025["Tax_Percentage"] = august_2025.apply(
    lambda row: (row["Tax"] / row["Amount"] * 100) 
    if pd.notnull(row["Tax"]) and row["Amount"] > 0 
    else 0,
    axis=1
).round(2)   # round to 2 decimals

# ==============================
# 📋 Preview Data
# ==============================
print("\n📋 SAMPLE WITH TAX PERCENTAGE:")
print(
    august_2025[
        ["Order ID", "Amount", "Tax", "Tax_Percentage", "Location", 
         "Transaction Date", "startdate", "Membership Type", "Match_Status"]
    ].head(10)
)

# ==============================
# 📊 Quick Summary
# ==============================
print("\n📊 TAX PERCENTAGE SUMMARY:")
print(august_2025["Tax_Percentage"].describe())



📋 SAMPLE WITH TAX PERCENTAGE:
           Order ID  Amount    Tax  Tax_Percentage        Location  \
0  MEM1753966592596  152.55    NaN            0.00  East Nashville   
1  MEM1756566329830  207.43  18.43            8.88  East Nashville   
2  MEM1756561841996  130.60  11.60            8.88  East Nashville   
3  MEM1753899349535  240.35    NaN            0.00  East Nashville   
4  MEM1753796245426  130.60    NaN            0.00  East Nashville   
5  MEM1756334515636  152.55  13.55            8.88  East Nashville   
6  MEM1753534775988  130.60    NaN            0.00  East Nashville   
7  MEM1753463742302  139.00    NaN            0.00             NaN   
8  MEM1753462253033  152.55    NaN            0.00  East Nashville   
9  MEM1753455083253  130.60    NaN            0.00  East Nashville   

  Transaction Date  startdate Membership Type Match_Status  
0       2025-08-31 2025-07-31       Recurring    Not Match  
1       2025-08-30 2025-08-30             New        Match  
2       2025-08

In [84]:
import numpy as np

# 1. Find the most common (mode) tax percentage from rows where Tax exists
common_tax_percentage = august_2025.loc[
    august_2025["Tax"].notnull(), "Tax_Percentage"
].mode()[0]

print(f"📌 Using most common tax percentage for missing taxes: {common_tax_percentage:.2f}%")

# 2. Create Final_Tax (fill missing Tax using common percentage)
august_2025["Final_Tax"] = august_2025.apply(
    lambda row: row["Tax"] if pd.notnull(row["Tax"]) 
                else (row["Amount"] * common_tax_percentage / 100) if pd.notnull(row["Amount"]) 
                else 0,
    axis=1
).round(2)

# 3. Summary of filled vs original
filled_count = august_2025["Tax"].isna().sum()
original_count = len(august_2025) - filled_count

print(f"✅ Final_Tax created: {original_count} kept from original, {filled_count} filled using {common_tax_percentage:.2f}%")

# 4. Show sample results
print("\n📋 SAMPLE DATA WITH FINAL_TAX:")
print(
    august_2025[
        ["Order ID", "Amount", "Tax", "Tax_Percentage", "Final_Tax", "Membership Type", "Match_Status"]
    ].head(10)
)


📌 Using most common tax percentage for missing taxes: 8.88%
✅ Final_Tax created: 22 kept from original, 7 filled using 8.88%

📋 SAMPLE DATA WITH FINAL_TAX:
           Order ID  Amount    Tax  Tax_Percentage  Final_Tax Membership Type  \
0  MEM1753966592596  152.55    NaN            0.00      13.55       Recurring   
1  MEM1756566329830  207.43  18.43            8.88      18.43             New   
2  MEM1756561841996  130.60  11.60            8.88      11.60             New   
3  MEM1753899349535  240.35    NaN            0.00      21.34       Recurring   
4  MEM1753796245426  130.60    NaN            0.00      11.60       Recurring   
5  MEM1756334515636  152.55  13.55            8.88      13.55             New   
6  MEM1753534775988  130.60    NaN            0.00      11.60       Recurring   
7  MEM1753463742302  139.00    NaN            0.00      12.34       Recurring   
8  MEM1753462253033  152.55    NaN            0.00      13.55       Recurring   
9  MEM1753455083253  130.60    NaN

In [83]:
# ==============================
# 📊 Membership Totals with Tax by Year & Month
# ==============================

# Extract year & month from Transaction Date
august_2025["Year"] = august_2025["Transaction Date"].dt.year
august_2025["Month"] = august_2025["Transaction Date"].dt.month

# Group by Year, Month, Membership Type
membership_summary = (
    august_2025
      .groupby(["Year", "Month", "Membership Type"])
      .agg(
          Total_Amount=("Amount", "sum"),
          Total_Tax=("Final_Tax", "sum")
      )
      .reset_index()
)

# Calculate effective tax percentage
membership_summary["Effective_Tax_%"] = (
    (membership_summary["Total_Tax"] / membership_summary["Total_Amount"]) * 100
).round(2)

# Add grand total row (for all Membership Types combined)
grand_totals = pd.DataFrame({
    "Year": ["All"],
    "Month": ["All"],
    "Membership Type": ["Grand Total"],
    "Total_Amount": [membership_summary["Total_Amount"].sum()],
    "Total_Tax": [membership_summary["Total_Tax"].sum()],
    "Effective_Tax_%": [
        (membership_summary["Total_Tax"].sum() / membership_summary["Total_Amount"].sum()) * 100
    ]
})

grand_totals["Effective_Tax_%"] = grand_totals["Effective_Tax_%"].round(2)

# Combine
membership_summary = pd.concat([membership_summary, grand_totals], ignore_index=True)

# Pretty print
print("\n📊 MEMBERSHIP SUMMARY (Year, Month, Amount + Tax)")
print(membership_summary.to_string(index=False))



📊 MEMBERSHIP SUMMARY (Year, Month, Amount + Tax)
Year Month Membership Type  Total_Amount  Total_Tax  Effective_Tax_%
2025     8             New       1394.91     123.91             8.88
2025     8       Recurring       3103.31     275.64             8.88
 All   All     Grand Total       4498.22     399.55             8.88
